<a href="https://colab.research.google.com/github/arobiddin/data_scince_ls_works/blob/main/05_ml_05_amaliyot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. Machine Learning

### Portfolio uchun vazifa: Toshkent shahrida uylarning narxini aniqlash.

Ushbu amaliyotda sizning vazifangiz berilgan ma`lumotlar asosida Toshkent shahridagi uylarning narxini aniqlash.

In [11]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.head()

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


# Ustunlar ta'rifi
- `location` - sotilayotgan uy manzili
- `district` - uy joylashgan tuman
- `rooms` - xonalar soni
- `size` - uy maydoni (kv.m)
- `level` - uy

## Vazifani CRSIP-DM Metolodgiyasi yordamida bajaring.
<img src="https://i.imgur.com/dzZnnYi.png" alt="CRISP-DM" width="800"/>

# 1.Faoliyatni o'rganish.
1.1 Jarayonni o'rganamiz.
Berilgan ma`lumotlar asosida Toshkent shahridagi uylarning narxini aniqlash, ya'ni berilgan qilymatga yaqin qiymat topuvchi(bashorat qiluvchi) model tuzish.

1.2 Analitik yondashuv.
1)Supervised learning - sababi bizda ma'lumotlarda label (yorliq) mavjud. Bu hududdadi median narx. Biz ham aynan shu narxni bashorat qilmoqchimiz.
2)Regressiya - "Bashorat" (prognoz) dedikmi demak bu regressia algoritmlari yordamida hal qilinadi
3)Offline - sababi ma'lumotlar bizga avvaldan bir marta berilgan. Doimiy ma'lumotlar oqimi yo'q. Offline bajarish imkoni bor lekin qulaylik uchun online bajaryapman.

# Ma'lumotlarni o'rganish

In [12]:
import numpy as np
import sklearn

In [13]:
sum(df.duplicated())

700

In [14]:
df = df.drop_duplicates() # Takrorlangan qiymatlarni o'chirib tashlaymiz.


In [15]:
dfx = df[df['price']=='Договорная'].index   # Nostandart qiymatlarni ham o'chirib tashlaymiz.
df.drop(dfx, inplace=True)

In [16]:
df['price'] = df['price'].astype(str).astype(int)


In [17]:
df[df['size']=='Площадьземли:1сот']['size']  # Ma'lumotlarni standart holatga keltiramiz
df.loc[5347, 'size']=100
df['size'] = df['size'].astype(str).astype(float)

In [18]:
# Grafik analizdan ayon bo'ldiki yuqori qiymatli uylar modelni yaratishga imkon yaratmaydi ekan.
df = df[(df['level']<=10) & (df['max_levels']<15) & (df['rooms']<=6) & (df['size']<200) &(df['price']<120_000)]


In [19]:
# Uylarni baholashda asosan tumanlarga e'tibor qaratildi chunki ma'lumotlar oz. Ma'lumotlar ko'paygan sari uy manzillari bilan ishlash ehtiyoji ko'payadi.  
df = df.drop('location', axis=1)
df.head()

,district,rooms,size,level,max_levels,price
0,Юнусабадский,3,57.0,4,4,52000
1,Яккасарайский,2,52.0,4,5,56000
2,Чиланзарский,2,42.0,4,4,37000
3,Чиланзарский,3,65.0,1,4,49500
4,Чиланзарский,3,70.0,3,5,55000


## 3. Ma'lumotlarni Machine Learning uchun tayyorlash

## Ma'lumotlarni train & testga ajratamiz

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=71)

X_train = train_set.drop("price", axis=1)
y = train_set["price"].copy()

X_num = X_train.drop('district', axis=1)
X_num

,rooms,size,level,max_levels
6997,5,135.0,2,9
4067,3,107.0,2,7
4627,3,85.0,7,9
2384,1,40.0,3,5
3194,2,54.0,4,4
...,...,...,...,...
1465,3,100.0,4,4
1779,3,60.0,3,9
6295,4,100.0,3,4
3599,2,60.0,3,9


## Pipeline quramiz

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# bizga kerak ustunlar indekslari
rooms_ix,	size_ix,	max_level_ix,	= 0, 1, 3

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_SLR = True):
        self.add_SLR = add_SLR
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        size_room = X[:, size_ix] / X[:, rooms_ix] 
        if self.add_SLR: # price_level ustuni ixtiyoriy bo'ladi
            size_level = X[:, size_ix] / X[:, max_level_ix] 
            return np.c_[X, size_room, size_level] 
        else:
            return np.c_[X, size_room] 


## Sonli ustunlar uchun 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

num_pipeline = Pipeline([
          ('imputer', SimpleImputer(strategy='median')),
          ('attribs_adder', CombinedAttributesAdder(add_SLR = True)),
          ('std_scaler', StandardScaler())             
])

## Matnli ustunlar uchun va full Pipeline

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ['district']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OrdinalEncoder(), cat_attribs)
])

In [ ]:
X_prepared = full_pipeline.fit_transform(X_train)

In [ ]:
X_prepared[0:5,:]

array([[ 2.52482918,  2.84441575, -0.76831035,  1.39238736, -0.11882076,
         0.40290478,  8.        ],
       [ 0.49445256,  1.6786572 , -0.76831035,  0.52099963,  1.16314315,
         0.45232679,  3.        ],
       [ 0.49445256,  0.76270405,  1.63886733,  1.39238736,  0.07840445,
        -0.55807865,  1.        ],
       [-1.53592406, -1.11083648, -0.28687482, -0.35038811,  1.80412511,
        -0.80793435,  8.        ],
       [-0.52073575, -0.5279572 ,  0.19456072, -0.78608198, -0.11882076,
         0.14343926,  1.        ]])

## Machine Learning

In [ ]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

In [ ]:
LR_model.fit(X_prepared, y)

LinearRegression()

## Modelni baholaymiz

In [ ]:
# tasodifiy 5 ta qatorni ajratib olamiz
test_data = X_train.sample(5)
test_data

,district,rooms,size,level,max_levels
3480,Чиланзарский,2,60.0,3,9
2650,Мирзо-Улугбекский,1,18.0,2,4
4931,Яккасарайский,2,53.0,5,8
263,Яшнободский,3,63.0,4,4
6,Чиланзарский,1,30.0,2,4


In [ ]:
test_label = y.loc[test_data.index]
test_label

3480    38000
2650    14000
4931    43000
263     40000
6       21200
Name: price, dtype: int64

In [ ]:
test_data_prepared = full_pipeline.transform(test_data)
test_data_prepared

array([[-0.52073575, -0.2781518 , -0.28687482,  1.39238736,  0.32493598,
        -1.03857037,  6.        ],
       [-1.53592406, -2.02678962, -0.76831035, -0.78608198, -1.45009099,
        -1.41335391,  2.        ],
       [-0.52073575, -0.56959144,  0.67599626,  0.95669349, -0.19278022,
        -1.04577775,  9.        ],
       [ 0.49445256, -0.1532491 ,  0.19456072, -0.78608198, -1.00633425,
         0.53263755, 11.        ],
       [-1.53592406, -1.52717882, -0.76831035, -0.78608198,  0.32493598,
        -0.89442286,  6.        ]])

In [ ]:
predicted_data = LR_model.predict(test_data_prepared)
predicted_data

array([48041.07684494, 19525.45646687, 38764.23170756, 42826.80597298,
       25565.80676021])

In [ ]:
pd.DataFrame({'Prognoz':predicted_data, 'Real baxosi': test_label})

,Prognoz,Real baxosi
3480,48041.076845,38000
2650,19525.456467,14000
4931,38764.231708,43000
263,42826.805973,40000
6,25565.806760,21200


In [ ]:
test_set

,district,rooms,size,level,max_levels,price
7460,Сергелийский,3,78.0,5,5,42000
532,Чиланзарский,2,51.0,3,5,24200
1586,Мирабадский,1,45.0,3,9,56500
6082,Яшнободский,3,70.0,3,4,42000
5970,Учтепинский,2,58.0,2,4,32000
...,...,...,...,...,...,...
2642,Мирзо-Улугбекский,2,47.0,4,4,27500
73,Мирзо-Улугбекский,3,95.0,5,5,50000
4888,Шайхантахурский,4,80.0,2,4,75000
5734,Чиланзарский,2,58.0,1,9,39500


In [ ]:
X_test = test_set.drop('price', axis=1)
X_test

,district,rooms,size,level,max_levels
7460,Сергелийский,3,78.0,5,5
532,Чиланзарский,2,51.0,3,5
1586,Мирабадский,1,45.0,3,9
6082,Яшнободский,3,70.0,3,4
5970,Учтепинский,2,58.0,2,4
...,...,...,...,...,...
2642,Мирзо-Улугбекский,2,47.0,4,4
73,Мирзо-Улугбекский,3,95.0,5,5
4888,Шайхантахурский,4,80.0,2,4
5734,Чиланзарский,2,58.0,1,9


In [ ]:
y_test = test_set['price'].copy()
y_test

7460    42000
532     24200
1586    56500
6082    42000
5970    32000
        ...  
2642    27500
73      50000
4888    75000
5734    39500
3191    22500
Name: price, Length: 1237, dtype: int64

In [ ]:
X_test_prepared = full_pipeline.transform(X_test)

In [ ]:
y_predicted = LR_model.predict(X_test_prepared)

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_predicted)
print("MAE=",mae)

MAE= 10082.005643329361


In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

RandomForestRegressor()

In [ ]:
from sklearn.metrics import mean_squared_error
y_predicted = RF_model.predict(X_test_prepared)
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

11827.956024332445


In [ ]:
X = df.drop("price", axis=1)
y = df["price"].copy()

X_prepared = full_pipeline.transform(X)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Std.dev:", scores.std())

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(RF_model, X_prepared, y, scoring="neg_mean_squared_error", cv=5)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [11159.8306643  13030.57435178 12514.60922013 11903.98839074
 11294.30827015]
Mean: 11980.66217942047
Std.dev: 712.4801676947245


## Joblib yordamida modelimizni saqlaymiz

In [ ]:
import joblib

filename = 'RF_model.jbl' # faylga istalgan nom beramiz
joblib.dump(RF_model, filename)

['RF_model.jbl']

In [ ]:
model = joblib.load(filename)
filename = 'pipeline.jbl'
joblib.dump(full_pipeline, filename)

['pipeline.jbl']